In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, row_number,sum,desc
from pyspark.sql import Window

## this is the expected output for the analytical questions.

In [29]:
spark = SparkSession.builder.appName("Analysis").getOrCreate()

In [30]:
# File location and type
file_location = "temp_dir/Data/Units_use.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_units = spark.read.format(file_type) \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("sep", delimiter) \
  .load(file_location)

In [31]:
# File location and type
file_location = "temp_dir/Data/Charges_use.csv"
file_type = "csv"

# CSV options
infer_schema = "false"

delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_charges = spark.read.format(file_type) \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("sep", delimiter) \
  .load(file_location)



In [32]:
# File location and type
file_location = "temp_dir/Data/Damages_use.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_damages = spark.read.format(file_type) \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("sep", delimiter) \
  .load(file_location)

In [33]:
# File location and type
file_location = "temp_dir/Data/Primary_Person_use.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_primary = spark.read.format(file_type) \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("sep", delimiter) \
  .load(file_location)



In [35]:
#analysis-1
conditions = (df_primary['PRSN_INJRY_SEV_ID'] == 'KILLED') & (df_primary['PRSN_GNDR_ID'] == 'MALE')
result = df_primary.filter(conditions) \
            .groupBy(['PRSN_INJRY_SEV_ID','PRSN_GNDR_ID']) \
            .agg(count('*').alias('cnt')) \
                .filter('cnt > 2')
result.show()

+-----------------+------------+---+
|PRSN_INJRY_SEV_ID|PRSN_GNDR_ID|cnt|
+-----------------+------------+---+
|           KILLED|        MALE|182|
+-----------------+------------+---+



In [36]:
#Analysis-2
motorcycle_count = df_units.filter(col("VEH_BODY_STYL_ID").contains("MOTORCYCLE"))
motorcycle_count.select('CRASH_ID').summary().limit(1).show()

+-------+--------+
|summary|CRASH_ID|
+-------+--------+
|  count|     784|
+-------+--------+



In [39]:
#Analysis 3 
joined=df_primary.join(df_units,df_units.CRASH_ID==df_primary.CRASH_ID)
conditions = (df_primary['PRSN_INJRY_SEV_ID'] == 'KILLED') & (df_primary['PRSN_AIRBAG_ID']=='NOT DEPLOYED') & (df_units['VEH_MAKE_ID'] !='NA')
filtered_2=joined.filter(conditions).groupBy('VEH_MAKE_ID').agg(count("*").alias("count")).orderBy(desc(count('*'))).limit(5)
filtered_2.show()

+------------+-----+
| VEH_MAKE_ID|count|
+------------+-----+
|   CHEVROLET|   17|
|        FORD|   12|
|       DODGE|    7|
|FREIGHTLINER|    6|
|      NISSAN|    5|
+------------+-----+



In [41]:
#analysis 4: 
df = (
            df_units.select("CRASH_ID", "VEH_HNR_FL")
            .join(
                df_primary.select("CRASH_ID", "DRVR_LIC_TYPE_ID"),
                on=["CRASH_ID"],
                how="inner"
            )
            .filter(
                (col("VEH_HNR_FL") == "Y")
                & (
                    col("DRVR_LIC_TYPE_ID").isin(
                        ["DRIVER LICENSE", "COMMERCIAL DRIVER LIC."]
                    )
                )
            )
        )

df.count()

6359

In [42]:
#analysis 5 
analysis_5 = df_primary.filter(df_primary['PRSN_GNDR_ID'] != 'FEMALE') \
                      .groupBy('DRVR_LIC_STATE_ID') \
                      .agg(count('*').alias('total_accidents')) \
                      .orderBy(desc(col('total_accidents'))) \
                      .limit(1)
analysis_5.show()

+-----------------+---------------+
|DRVR_LIC_STATE_ID|total_accidents|
+-----------------+---------------+
|            Texas|          83016|
+-----------------+---------------+



In [43]:
#analysis 6 
windowSpec = Window.orderBy(col("total_injuries").desc())
# Perform the analysis
analysis_6 = df_units.groupBy("VEH_MAKE_ID") \
                   .agg(sum(col("TOT_INJRY_CNT") + col("DEATH_CNT")).alias("total_injuries")) \
                   .select("VEH_MAKE_ID", "total_injuries", row_number().over(windowSpec).alias("row_number"))
# Filter rows where row number falls within the range of 3 to 5
result_6 = analysis_6.filter((col("row_number") >= 3) & (col("row_number") <= 5)).drop("row_number")
result_6.show()

24/06/01 17:38:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/01 17:38:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/01 17:38:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/01 17:38:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/01 17:38:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/01 17:38:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+--------------+
|VEH_MAKE_ID|total_injuries|
+-----------+--------------+
|     TOYOTA|          4228|
|      DODGE|          3146|
|     NISSAN|          3118|
+-----------+--------------+



In [44]:
grouped_df = joined.groupBy('VEH_BODY_STYL_ID', 'PRSN_ETHNICITY_ID') \
    .agg(count('*').alias('ethnicity_count'))

# Find the top ethnicity for each body style
window_spec = Window.partitionBy('VEH_BODY_STYL_ID').orderBy(desc('ethnicity_count'))
top_ethnicity_df = grouped_df.withColumn('rank', row_number().over(window_spec)) \
    .filter(col('rank') == 1) \
    .select('VEH_BODY_STYL_ID', 'PRSN_ETHNICITY_ID', 'ethnicity_count') \
    .orderBy('VEH_BODY_STYL_ID')

# Display the results
top_ethnicity_df.show(truncate=False)

+---------------------------------+-----------------+---------------+
|VEH_BODY_STYL_ID                 |PRSN_ETHNICITY_ID|ethnicity_count|
+---------------------------------+-----------------+---------------+
|AMBULANCE                        |WHITE            |97             |
|BUS                              |HISPANIC         |391            |
|FARM EQUIPMENT                   |WHITE            |63             |
|FIRE TRUCK                       |WHITE            |112            |
|MOTORCYCLE                       |WHITE            |848            |
|NA                               |WHITE            |5693           |
|NEV-NEIGHBORHOOD ELECTRIC VEHICLE|WHITE            |10             |
|NOT REPORTED                     |HISPANIC         |2              |
|OTHER  (EXPLAIN IN NARRATIVE)    |WHITE            |459            |
|PASSENGER CAR, 2-DOOR            |WHITE            |9877           |
|PASSENGER CAR, 4-DOOR            |WHITE            |58312          |
|PICKUP             

In [45]:

#analysis 8 
df = (
    df_units.join(df_primary, on=["CRASH_ID"], how="inner")
    .dropna(subset=["DRVR_ZIP"])
    .filter(
        col("CONTRIB_FACTR_1_ID").contains("ALCOHOL")
        | col("CONTRIB_FACTR_2_ID").contains("ALCOHOL")
    )
    .groupby("DRVR_ZIP")
    .count()
    .orderBy(col("count").desc())
    .limit(5)
)
df.show()


+--------+-----+
|DRVR_ZIP|count|
+--------+-----+
|   76010|   75|
|   78521|   61|
|   75067|   54|
|   78574|   47|
|   75052|   43|
+--------+-----+



In [49]:
df = (
    df_damages.join(df_units, on=["CRASH_ID"], how="inner")
    .filter(
        (
            (df_units.VEH_DMAG_SCL_1_ID > "DAMAGED 4")
            & (
                ~df_units.VEH_DMAG_SCL_1_ID.isin(
                    ["NA", "NO DAMAGE", "INVALID VALUE"]
                )
            )
        )
        | (
            (df_units.VEH_DMAG_SCL_2_ID > "DAMAGED 4")
            & (
                ~df_units.VEH_DMAG_SCL_2_ID.isin(
                    ["NA", "NO DAMAGE", "INVALID VALUE"]
                )
            )
        )
    )
    .filter(df_damages.DAMAGED_PROPERTY == "NONE")
    .filter(df_units.FIN_RESP_TYPE_ID == "PROOF OF LIABILITY INSURANCE")
)

[row[0] for row in df.collect()]

[14870169, 14894076, 14996273, 15232090, 15232090, 15249931, 15307513]

In [50]:
top_25_state_list = [
row[0]
for row in df_units.filter(
    col("VEH_LIC_STATE_ID").cast("int").isNull()
)
.groupby("VEH_LIC_STATE_ID")
.count()
.orderBy(col("count").desc())
.limit(25)
.collect()
]
top_10_used_vehicle_colors = [
row[0]
for row in df_units.filter(df_units.VEH_COLOR_ID != "NA")
.groupby("VEH_COLOR_ID")
.count()
.orderBy(col("count").desc())
.limit(10)
.collect()
]

df = (
df_charges.join(df_primary, on=["CRASH_ID"], how="inner")
.join(df_units, on=["CRASH_ID"], how="inner")
.filter(df_charges.CHARGE.contains("SPEED"))
.filter(
    df_primary.DRVR_LIC_TYPE_ID.isin(
        ["DRIVER LICENSE", "COMMERCIAL DRIVER LIC."]
    )
)
.filter(df_units.VEH_COLOR_ID.isin(top_10_used_vehicle_colors))
.filter(df_units.VEH_LIC_STATE_ID.isin(top_25_state_list))
.groupby("VEH_MAKE_ID")
.count()
.orderBy(col("count").desc())
.limit(5)
)

[row[0] for row in df.collect()]

['FORD', 'CHEVROLET', 'TOYOTA', 'DODGE', 'NISSAN']